# Pre-processing Utah Allocation data for WaDEQA upload.
- Purpose:  To pre-process the Utah data into one master file for simple DataFrame creation and extraction

In [ ]:
#Needed Libararies
import os
import re
import numpy as np
import pandas as pd
import geopandas as gpd # the library that lets us read in shapefiles
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebooki

# Working Directory
workingDir = "G:/Shared drives/WaDE Data/Utah/WaterAllocation/RawInputData"
os.chdir(workingDir)

from pyproj import Transformer, transform
transformer = Transformer.from_proj(26912, 4326)  # A trick to drastically optimize the Transformer of pyproj.
# Utah projection = EPSG:26912.  WGS84 projection used by WaDE 2.0 = epsg:4326.

## Point of Diversion Data

In [ ]:
# Input Files
FI_PoD = "Point of Diversion/PointsOfDiversion_input.csv"
FI_WMs = "Point of Diversion/WRCHEX_WATER_MASTER.csv"
FI_Irr = "Point of Diversion/IRRIGATION_MASTER.csv"
FI_Mun = "Point of Diversion/WTRUSE_MUNICIPAL.csv"
FI_Pow = "Point of Diversion/WTRUSE_POWER.csv"

In [ ]:
# Dataframe creation
dfPODin = pd.read_csv(FI_PoD, encoding = "ISO-8859-1") # Point of Diversion Input
dfWMain = pd.read_csv(FI_WMs, encoding = "ISO-8859-1") # Irrigation Input
dfIrrin = pd.read_csv(FI_Irr, encoding = "ISO-8859-1") # Irrigation Input
dfMunin = pd.read_csv(FI_Mun, encoding = "ISO-8859-1") # Municiplal Input
dfPowin = pd.read_csv(FI_Pow, encoding = "ISO-8859-1") # Power Inpu

In [ ]:
# Merging dataframes into one, using left-join.
dfPODin = pd.merge(dfPODin, dfWMain[['WRNUM', 'DATE_FILED', 'DATE_TERMINATED', 'IRRIGATION_DEPLETION']], on='WRNUM', how='left')
dfPODin = pd.merge(dfPODin, dfIrrin[['WRNUM', 'USE_END_DATE', 'USE_BEG_DATE']], on='WRNUM', how='left')
dfPODin = pd.merge(dfPODin, dfMunin[['WRNUM', 'MUNICIPALITY']], on='WRNUM', how='left')
dfPODin = pd.merge(dfPODin, dfPowin[['WRNUM', 'POWER_CAPACITY']], on='WRNUM', how='left')
dfPODin = dfPODin.replace(np.nan, "").reset_index()
print(len(dfPODin))
dfPODin.head(1)

In [ ]:
# For creating BeneficialUseCategory
benUseDict = {
    "I" : "Irrigation",
    "S" : "Stockwatering",
    "D" : "Domestic",
    "M" : "Municipal",
    "X" : "Mining",
    "P" : "Power",
    "O" : "Other"}
def assignBenUseCategory(colrowValue):
    colrowValue = str(colrowValue).strip()
    if colrowValue == "" or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        outList = ",".join(benUseDict[inx] for inx in list(str(colrowValue)))
    return outList


dfPODin['in_BeneficialUseCategory'] = dfPODin.apply(lambda row: assignBenUseCategory(row['USES']), axis=1)
dfPODin['in_BeneficialUseCategory'].unique()

In [ ]:
# Clean Owner info. Remove special characters
import re

def cleanOwnerDataFunc(Val):
    Val = re.sub("[$@&.;,/\)(-]", "", Val).strip()
    return Val

dfPODin['OWNER'] = dfPODin.apply(lambda row: cleanOwnerDataFunc(row['OWNER']), axis=1)
dfPODin['OWNER'].unique()

In [ ]:
def fixNativeID(valA):
    outString = str(valA).strip()
    return outString

dfPODin['WRNUM'] = dfPODin.apply(lambda row: fixNativeID(row['WRNUM']), axis=1)

In [ ]:
# Creating the output Dataframe for PODs.

dfPOD = pd.DataFrame(index=dfPODin.index)

# Water Source
dfPOD["in_WaterSourceTypeCV"] = dfPODin['TYPE']

# Site
dfPOD["in_Latitude"] = dfPODin['Latitude']
dfPOD["in_Longitude"] = dfPODin['Longitude']
dfPOD["in_PODorPOUSite"] = "POD"
dfPOD["in_SiteName"] = dfPODin['SOURCE']
dfPOD["in_SiteNativeID"] = "POD" + dfPODin['OBJECTID'].astype(str)
dfPOD["in_SiteTypeCV"] = dfPODin['SOURCE']

# Allocation
dfPOD['in_AllocationApplicationDate'] = dfPODin['DATE_FILED']
dfPOD['in_AllocationCommunityWaterSupplySystem'] = dfPODin['MUNICIPALITY']
dfPOD["in_AllocationCropDutyAmount"] = dfPODin['IRRIGATION_DEPLETION']
dfPOD["in_AllocationExpirationDate"] = dfPODin['DATE_TERMINATED']
dfPOD["in_AllocationFlow_CFS"] = dfPODin['CFS'].astype(float)
dfPOD['in_AllocationLegalStatusCV'] = dfPODin['STATUS']
dfPOD["in_AllocationNativeID"] = dfPODin['WRNUM']
dfPOD['in_AllocationOwner'] = dfPODin['OWNER']
dfPOD['in_AllocationPriorityDate'] = dfPODin['PRIORITY']
dfPOD["in_AllocationVolume_AF"] = dfPODin['ACFT'].astype(float)
dfPOD["in_BeneficialUseCategory"] = dfPODin['in_BeneficialUseCategory']  #see above
dfPOD['USE_BEG_DATE'] = dfPODin['USE_BEG_DATE']
dfPOD['USE_END_DATE'] = dfPODin['USE_END_DATE']

dfPOD = dfPOD.drop_duplicates().reset_index(drop=True)
print(len(dfPOD))
dfPOD.head(1)

## Place of Use Data

In [ ]:
# Input Files
FI_POU = "Place of Use/PlaceOfUseService_input.csv"

In [ ]:
# Dataframe creation
dfPOUin = pd.read_csv(FI_POU, encoding = "ISO-8859-1") # Place of Use Input
print(len(dfPOUin))
dfPOUin.head(1)

In [ ]:
# Need to split out WRNUMS into their own row
# The explode() method explodes lists into separate rows.
dfPOUin = dfPOUin.assign(WRNUMS=dfPOUin['WRNUMS'].str.split(',')).explode('WRNUMS').reset_index()
dfPOUin = dfPOUin.rename({'WRNUMS': 'WRNUM'}, axis=1)
dfPOUin = dfPOUin.replace(np.nan, "").reset_index()
print(len(dfPOUin))
dfPOUin.head(1)

In [ ]:
# Merging dataframes into one, using left-join.
dfPOUin = pd.merge(dfPOUin, dfWMain[['WRNUM', 'DATE_FILED', 'DATE_TERMINATED', 'IRRIGATION_DEPLETION']], on='WRNUM', how='left')
dfPOUin = pd.merge(dfPOUin, dfIrrin[['WRNUM', 'USE_END_DATE', 'USE_BEG_DATE']], on='WRNUM', how='left')
dfPOUin = pd.merge(dfPOUin, dfMunin[['WRNUM', 'MUNICIPALITY']], on='WRNUM', how='left')
dfPOUin = pd.merge(dfPOUin, dfPowin[['WRNUM', 'POWER_CAPACITY']], on='WRNUM', how='left')
print(len(dfPOUin))
dfPOUin.head(1)

In [ ]:
def fixNativeID(valA):
    outString = str(valA).strip()
    return outString

dfPOUin['WRNUM'] = dfPOUin.apply(lambda row: fixNativeID(row['WRNUM']), axis=1)

In [ ]:
# CFS and AF not provided for POU data.  Will instead assume they share values.
# merging dfPOD data to ensure that the POUs are using the same CFS and AF as the PODS.
dfPOUin = pd.merge(dfPOUin, dfPOD, left_on='WRNUM', right_on='in_AllocationNativeID', how='left')
print(len(dfPOUin))
dfPOUin.head(1)

In [ ]:
# Creating the output Dataframe for POUs.

dfPOU = pd.DataFrame(index=dfPOUin.index)

# Water Source
dfPOU["in_WaterSourceTypeCV"] = ""

# Site
dfPOU["in_Latitude"] = dfPOUin['Latitude']
dfPOU["in_Longitude"] = dfPOUin['Longitude']
dfPOU["in_PODorPOUSite"] = "POU"
dfPOU["in_SiteName"] = "Unspecified"
dfPOU["in_SiteNativeID"] = "POU" + dfPOUin['RECORD_ID'].astype(str)
dfPOU["in_SiteTypeCV"] = ""

# Allocation
dfPOU['in_AllocationApplicationDate'] = ""
dfPOU['in_AllocationCommunityWaterSupplySystem'] = ""
dfPOU["in_AllocationCropDutyAmount"] = ""
dfPOU["in_AllocationExpirationDate"] = ""
dfPOU["in_AllocationFlow_CFS"] = dfPOUin['in_AllocationFlow_CFS'].astype(float)  # from POD data
dfPOU['in_AllocationLegalStatusCV'] = ""
dfPOU["in_AllocationNativeID"] = dfPOUin['WRNUM']
dfPOU['in_AllocationOwner'] = dfPOUin['in_AllocationOwner']  # from POD data
dfPOU['in_AllocationPriorityDate'] = dfPOUin['in_AllocationPriorityDate']  # from POD data
dfPOU["in_AllocationVolume_AF"] = dfPOUin['in_AllocationVolume_AF'].astype(float)  # from POD data
dfPOU["in_BeneficialUseCategory"] = dfPOUin['in_BeneficialUseCategory']  # from POD data
dfPOU['USE_BEG_DATE'] = ""
dfPOU['USE_END_DATE'] = ""

dfPOU = dfPOU.drop_duplicates().reset_index(drop=True)
print(len(dfPOUin))
dfPOU.head(1)

## Concatenate POD and POU Data

In [ ]:
# Concatenate
frames = [dfPOD, dfPOU]
dfout = pd.concat(frames)

#Removing all NaN Values and replacing with blank
dfout = dfout.replace(np.nan, "", regex=True).reset_index()

print(len(dfout))
dfout.head(1)

In [ ]:
# Making Sure datatype of Long, Lat, Wrex, Irrigation are Float
dfout['in_AllocationFlow_CFS'] = pd.to_numeric(dfout['in_AllocationFlow_CFS'], errors='coerce')
dfout['in_AllocationVolume_AF'] = pd.to_numeric(dfout['in_AllocationVolume_AF'], errors='coerce')
dfout['in_AllocationCropDutyAmount'] = pd.to_numeric(dfout['in_AllocationCropDutyAmount'], errors='coerce')

In [ ]:
# Changing datatype of used date fields. 
dfout['in_AllocationPriorityDate'] = pd.to_datetime(dfout['in_AllocationPriorityDate'], errors = 'coerce')
dfout['in_AllocationPriorityDate'] = pd.to_datetime(dfout["in_AllocationPriorityDate"].dt.strftime('%m/%d/%Y'))

dfout['in_AllocationApplicationDate'] = pd.to_datetime(dfout['in_AllocationApplicationDate'], errors = 'coerce')
dfout['in_AllocationApplicationDate'] = pd.to_datetime(dfout["in_AllocationApplicationDate"].dt.strftime('%m/%d/%Y'))

dfout['in_AllocationExpirationDate'] = pd.to_datetime(dfout['in_AllocationExpirationDate'], errors = 'coerce')
dfout['in_AllocationExpirationDate'] = pd.to_datetime(dfout["in_AllocationExpirationDate"].dt.strftime('%m/%d/%Y'))

In [ ]:
# Creating WaterSourceTypeCV

WaterSourceTypeCVDictionary={
"Underground" : "Groundwater",
"Abandonded Well" : "Groundwater",
"Point to Point" : "Surface Water",
"Surface" : "Surface Water",
"Return" : "Surface Water",
"Drain" : "Surface Water",
"Spring" : "Surface Water",
"Rediversion" : "Surface Water"}
def CreateWaterSourceTypeCV(val):
    if val == "" or pd.isnull(val):
        outString = "Unspecified"
    else:
        val = val.strip()
        try:
            outString = WaterSourceTypeCVDictionary[val]
        except:
            outString = "Unspecified"
    return outString

dfout['in_WaterSourceTypeCV'] = dfout.apply(lambda row: CreateWaterSourceTypeCV(row['in_WaterSourceTypeCV']), axis=1)
dfout['in_WaterSourceTypeCV'].unique()

In [ ]:
#Compiling 'AllocationTimeframeStart' & 'AllocationTimeframeEnd'
#Both can have a string format for WaDE 2.0.
    
def assignTime(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outlist = ""
    else:
        colrowValue = str(colrowValue)
        colrowValue = colrowValue.strip()
        if len(colrowValue) == 4:
            startmonth = colrowValue[0:1]
            startday = colrowValue[1:2]
            outlist = "0" + startmonth + "/" + "0" + startday
        elif len(colrowValue) == 5:
            startmonth = colrowValue[0:1]
            startday = colrowValue[1:3]
            outlist = "0" + startmonth + "/" + startday
        elif len(colrowValue) == 6:
            startmonth = colrowValue[0:2]
            startday = colrowValue[2:4]
            outlist = startmonth + "/" + startday
        else:
            outlist = ""
  
    return outlist


dfout['in_AllocationTimeframeStart'] = dfout.apply(lambda row: assignTime(row['USE_BEG_DATE']), axis=1)
dfout['in_AllocationTimeframeEnd'] = dfout.apply(lambda row: assignTime(row['USE_END_DATE']), axis=1)

In [ ]:
# Assign SiteTypeCV value.
# Uses the re library, but requires for loop.
# Order that the lists are inputed into dictoinary is important, want to overide generic search with a more specific search.

# Create the Lists
canalList = ["canal", "canals"]
creekList = ["creek"]
ditchList = ["ditch"]
drainList = ["drain", "drains"]
lakeList = ["lake"]
pondList = ["pond"]
reservoirList = ["reservoir"]
riverList = ["river", "fork", "surface"]
sloughList = ["slough"]
springList = ["spring", "springs", "gulch", "seep"]
tunnelList = ["tunnel", "tunnels"]
washList = ["wash"]
wellList = ["well", "wells", "well:", "draw", "hollow"]

# Making the dictionary
listDictionary = {}
listDictionary["Canal"] = canalList
listDictionary["Creek"] = creekList
listDictionary["Ditch"] = ditchList
listDictionary["Drain"] = drainList
listDictionary["Lake"] = lakeList
listDictionary["Pond"] = pondList
listDictionary["Reservoir"] = reservoirList
listDictionary["River"] = riverList
listDictionary["Slough"] = sloughList
listDictionary["Spring"] = springList
listDictionary["Tunnel"] = tunnelList
listDictionary["Wash"] = washList
listDictionary["Well"] = wellList

def CreateSiteTypeCV(val):
    if val == "" or pd.isnull(val):
        outString = "Unspecified"
    else:
        outString = "Unspecified" # Default
        
        # Cleaning text / simple search format
        val = val.replace(",", " ")
        val = val.replace(".", " ")
        val = val.replace(";", " ")
        val = val.replace("-", " ")
        val = val.replace("/", " ")
        val = val.replace("(", " ")
        val = val.replace(")", " ")
        val = val.lower().strip()
        val = " "+val+" "
        
        for x in listDictionary:
            labelString = x
            valueList = listDictionary[x]
            for words in valueList:
                if re.search(" "+words+ " ", val): outString = x
            
    return outString

dfout['in_SiteTypeCV'] = dfout.apply(lambda row: CreateSiteTypeCV( row['in_SiteTypeCV']), axis=1)
dfout['in_SiteTypeCV'].unique()

In [ ]:
# Assign LegalStatusCV value.
# Uses the re library, but requires for loop.
# Order that the lists are inputed into dictoinary is important, want to overide generic search with a more specific search.

# Create the Lists
ADECList = ["ADEC"]
ADVList = ["ADV"]
APPList = ["APP"]
CERTList = ["CERT"]
DECList = ["DEC"]
DILList = ["DIL"]
DISList = ["DIS"]
EXPList = ["EXP"]
FORFList = ["FORF"]
LAPList = ["LAP"]
NPRList = ["NPR"]
NUSEList = ["NUSE"]
PERFList = ["PERF"]
REJList = ["REJ"]
RNUMList = ["RNUM"]
STATUSList = ["STATUS"]
TEMPList = ["TEMP"]
TERMList = ["TERM"]
UGWCList = ["UGWC"]
UNAPList = ["UNAP"]
WDList = ["WD"]
WUCList = ["WUC"]


# Making the dictionary
listDictionary = {}

listDictionary["Lapsed"] = LAPList

listDictionary["Adjudication Decree"] = ADECList
listDictionary["Adverse Use Claim"] = ADVList
listDictionary["Approved"] = APPList
listDictionary["Certificated"] = CERTList
listDictionary["Decree"] = DECList
listDictionary["Diligence Claim"] = DILList
listDictionary["Disallowed"] = DISList
listDictionary["Expired"] = EXPList
listDictionary["Forfeited"] = FORFList
listDictionary["No Proof Required"] = NPRList
listDictionary["Nonuse"] = NUSEList
listDictionary["Perfected"] = PERFList
listDictionary["Rejected"] = REJList
listDictionary["Renumbered"] = RNUMList
listDictionary["Deff"] = STATUSList
listDictionary["Temp Applications"] = TEMPList
listDictionary["Terminated"] = TERMList
listDictionary["Underground Water Claim"] = UGWCList
listDictionary["Unapproved"] = UNAPList
listDictionary["Withdrawn"] = WDList
listDictionary["Water User`s Claim"] = WUCList


def CreateLegalStatus(val):
    val = str(val).strip()
    if val == "" or pd.isnull(val):
        outString = ""
    else:
        outString = ""
        for x in listDictionary:
            valueList = listDictionary[x]
            for words in valueList:
                if words in val: outString = x

    return outString

dfout['in_AllocationLegalStatusCV'] = dfout.apply(lambda row: CreateLegalStatus( row['in_AllocationLegalStatusCV']), axis=1)
dfout['in_AllocationLegalStatusCV'].unique()

## WaDE Custom Elements (due to missing state site info)

In [ ]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEUT_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = dfout['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    if (A == '') or (pd.isnull(A)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceTypeCV'] == A), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfout['in_WaterSourceNativeID'] = dfout.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceTypeCV']), axis=1)
dfout['in_WaterSourceNativeID'].unique()

## Shapefile Data
- For attaching geometry to POU csv inputs.

In [ ]:
# PoU Shapefile Data
# Shapefile input
dfPoUshapetemp = gpd.read_file('Place of Use/PlaceOfUseService/PlaceOfUseService2.shp')
dfPoUshapetemp.head(3)

In [ ]:
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)
dfPoUshape['in_SiteNativeID'] = "POU" + dfPoUshapetemp['RECORD_ID'].astype(str)
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfPoUshape.head(3)

## Export Data

In [ ]:
#Removing all NaN Values and replacing with blank
dfout = dfout.replace(np.nan, "", regex=True)
print(len(dfout))
dfout.head(3)

In [ ]:
#Exporting to Finished File
dfout.to_csv('P_UtahMaster.csv', index=False)  # The output
dfPoUshape.to_csv('P_utGeometry.csv', index=False) # The output geometry.